# Water Demand Tutorial Phase - not of value

In [ ]:
%matplotlib inline

import os
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from cartopy.feature import ShapelyFeature
import cartopy.crs as ccrs
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import folium
import scipy
import seaborn as sns

# Set Seaborn style and color palette
sns.set_style("darkgrid")
sns.set_palette("husl")

In [ ]:
# generate matplotlib handles to create a legend of the features we put in our map.
def generate_handles(labels, colors, edge='k', alpha=1):
    lc = len(colors)  # get the length of the color list
    handles = []
    for i in range(len(labels)):
        handles.append(mpatches.Rectangle((0, 0), 1, 1, facecolor=colors[i % lc], edgecolor=edge, alpha=alpha))
    return handles


# create a scale bar of length 20 km in the upper right corner of the map
# adapted this question: https://stackoverflow.com/q/32333870
# answered by SO user Siyh: https://stackoverflow.com/a/35705477
def scale_bar(ax, location=(0.92, 0.95)):
    x0, x1, y0, y1 = ax.get_extent()
    sbx = x0 + (x1 - x0) * location[0]
    sby = y0 + (y1 - y0) * location[1]

    ax.plot([sbx, sbx - 20000], [sby, sby], color='k', linewidth=9, transform=ax.projection)
    ax.plot([sbx, sbx - 10000], [sby, sby], color='k', linewidth=6, transform=ax.projection)
    ax.plot([sbx-10000, sbx - 20000], [sby, sby], color='w', linewidth=6, transform=ax.projection)

    ax.text(sbx, sby-4500, '20 km', transform=ax.projection, fontsize=8)
    ax.text(sbx-12500, sby-4500, '10 km', transform=ax.projection, fontsize=8)
    ax.text(sbx-24500, sby-4500, '0 km', transform=ax.projection, fontsize=8) 

In [ ]:
# load the outline of UK for a backdrop
#outline = gpd.read_file(os.path.abspath('data_files/Outline.shp')) # if this is uncommented, the outline polygons are overlayed on top of the land use so they aren't visible

In [ ]:
#add scalebar
#add figure heading

In [ ]:
# Load water company data as wrz, remove unnecessary columns
wrz = gpd.read_file(os.path.abspath('data_files/WaterSupplyAreas_incNAVs v1_4.shp'))
# List of columns to be removed
columns_to_remove = ['Disclaimer', 'Disclaim2', 'Disclaim3', 'Provenance', 'Licence', 'WARNINGS', 'Revisions']

# Drop the columns from the GeoDataFrame
wrz = wrz.drop(columns=columns_to_remove)

In [ ]:
myFig = plt.figure(figsize=(10, 30))  # create a figure of size 10x30 (adjusted from 10x10 to accommodate the legend)

myCRS = ccrs.TransverseMercator(27700)  # create a Universal Transverse Mercator reference system to transform our data.

ax = plt.axes(projection=myCRS)  # finally, create an axes object in the figure, using a UTM projection,

In [ ]:
# first, we just add the outline using cartopy's ShapelyFeature
outline_feature = ShapelyFeature(outline['geometry'], myCRS, edgecolor='k', facecolor='w')
xmin, ymin, xmax, ymax = outline.total_bounds
ax.add_feature(outline_feature) # add the features created to the map.

In [ ]:
# using the boundary of the shapefile features, zoom the map to our area of interest
ax.set_extent([xmin-5000, xmax+5000, ymin-5000, ymax+5000], crs=myCRS) # because total_bounds 
# gives output as xmin, ymin, xmax, ymax,
# but set_extent takes xmin, xmax, ymin, ymax, we re-order the coordinates here.

#myFig ## uncomment if you would like to see a preview of the map

In [ ]:
#looking at land use   

In [ ]:
#read in land use data 

# Load landuse data
landuse = gpd.read_file(os.path.abspath('data_files/clc2018_uk.shp'))

In [ ]:
landuse.loc[1] #uncomment this to see the first record in the geodataframe

In [ ]:
# Load the CSV file
landuse_categories = pd.read_csv('data_files/legend.csv')
# print(landuse_categories.head())  #show a sample of the CSV file 

In [ ]:
# merge the csv file with the geodataframe to include the labels for the landuse in the geodataframe
landuse_categories['CODE'] = landuse_categories['CODE'].astype(str)
merged_landuse = pd.merge(landuse, landuse_categories, left_on='CODE_18', right_on='CODE')

# Drop unnecessary columns - this cleans the dataset to make it easier to work with
merged_landuse = merged_landuse.drop(['CODE_18', 'CODE', 'Unnamed: 4', 'Unnamed: 5'], axis=1)
merged_landuse
# Access the 'LABEL' column in the merged DataFrame - LABEL gives the actual landuse description
label_column = merged_landuse['LABEL']

In [ ]:
merged_landuse.loc[1]  #show the first row of the merged landuse geodataframe
merged_landuse.to_file('data_files/merged_landuse.shp', index=False) #save a copy of the geodataframe

In [ ]:
# get the number of unique landuse categories we have in the dataset
num_luse = len(merged_landuse.LABEL.unique())
print('Number of unique categories: {}'.format(num_luse))

# pick colours for each landuse colour - generate random RGB colours
num_lcolours = num_luse  # Number of colours to generate
luse_colours = sns.color_palette("husl", n_colors=num_lcolours)  # Use Seaborn colour palette

#get a list of unique names for landuse
landuse = list(merged_landuse.LABEL.unique())
landuse.sort() # sort landuse alphabetically by name

In [ ]:
print(merged_landuse.crs == outline.crs) # test if the crs is the same 

In [ ]:
# Define the colour map
my_cmap = 'viridis'  # Replace with the desired color map name, e.g., 'viridis', 'cividis', 'coolwarm', etc.

# Plot the 'merged_landuse' GeoDataFrame on the existing map
merged_landuse.plot(column='LABEL', cmap=my_cmap, linewidth=0.5, edgecolor='k', alpha=0.25, ax=ax)

In [ ]:
# Add vertical gridlines
ax.vlines(x=[100000, 200000, 300000, 400000, 500000, 600000, 700000],
          ymin=ax.get_ylim()[0], ymax=ax.get_ylim()[1],
          color='gray', linestyle=':', linewidth=0.5)

# Add horizontal gridlines
ax.hlines(y=[100000, 200000, 300000, 400000, 500000, 600000, 700000, 800000, 900000, 1000000, 1100000, 1200000],
          xmin=ax.get_xlim()[0], xmax=ax.get_xlim()[1],
          color='gray', linestyle=':', linewidth=0.5)

# Add labels to x-axis
x_ticks = [100000, 200000, 300000, 400000, 500000, 600000, 700000]
for x in x_ticks:
    ax.text(x, ax.get_ylim()[0] - 50000, str(x), ha='center', va='top', fontsize=8)

# Add labels to y-axis
y_ticks = [100000, 200000, 300000, 400000, 500000, 600000, 700000, 800000, 900000, 1000000, 1100000, 1200000]
for y in y_ticks:
    ax.text(ax.get_xlim()[0] - 50000, y, str(y), ha='right', va='center', fontsize=8)

ax.set_xlabel('X Axis Label')
ax.set_ylabel('Y Axis Label')

ax.set_title('Land Use for 2018') # Add a title to the figure

In [ ]:
myFig # to show the updated figure

In [1]:
import os
import random

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from cartopy.feature import ShapelyFeature
import cartopy.crs as ccrs
import matplotlib.patches as mpatches

import seaborn as sns


def generate_handles(labels, colors, edge='k', alpha=1):
    """Generate matplotlib handles to create a legend of the features we put in our map."""
    lc = len(colors)
    handles = []
    for i in range(len(labels)):
        handles.append(mpatches.Rectangle((0, 0), 1, 1, facecolor=colors[i % lc], edgecolor=edge, alpha=alpha))
    return handles


def scale_bar(ax, location=(0.92, 0.95)):
    """Create a scale bar of length 20 km in the upper right corner of the map."""
    x0, x1, y0, y1 = ax.get_extent()
    sbx = x0 + (x1 - x0) * location[0]
    sby = y0 + (y1 - y0) * location[1]

    ax.plot([sbx, sbx - 20000], [sby, sby], color='k', linewidth=9, transform=ax.projection)
    ax.plot([sbx, sbx - 10000], [sby, sby], color='k', linewidth=6, transform=ax.projection)
    ax.plot([sbx-10000, sbx - 20000], [sby, sby], color='w', linewidth=6, transform=ax.projection)

    ax.text(sbx, sby-4500, '20 km', transform=ax.projection, fontsize=8)
    ax.text(sbx-12500, sby-4500, '10 km', transform=ax.projection, fontsize=8)
    ax.text(sbx-24500, sby-4500, '0 km', transform=ax.projection, fontsize=8)


def main():
    """Main function to load data and create the map."""
    # Load the outline of UK for a backdrop
    outline_gdf = gpd.read_file(os.path.abspath('data_files/Outline.shp'))

    # Load water company data as wrz, remove unnecessary columns
    wrz_gdf = gpd.read_file(os.path.abspath('data_files/WaterSupplyAreas_incNAVs v1_4.shp'))

    # List of columns to be removed
    columns_to_remove = ['Disclaimer', 'Disclaim2', 'Disclaim3', 'Provenance', 'Licence', 'WARNINGS', 'Revisions']

    # Drop the columns from the GeoDataFrame
    wrz_gdf = wrz_gdf.drop(columns=columns_to_remove)

    # Create a Universal Transverse Mercator reference system to transform our data.
    myCRS = ccrs.TransverseMercator(27700)

    # Create a figure of size 10x10 (representing the page size
